文本分割器 [Text Splitters](https://python.langchain.com/docs/concepts/text_splitters/)的作用在于将大文本分割成小文本，这样有几个好处：

* 处理非均匀文档长度：现实世界的文档集合通常包含不同大小的文本。拆分可确保对所有文档进行一致的处理；
* 克服模型限制：许多嵌入模型和语言模型都有最大输入大小限制。拆分使我们能够处理超出这些限制的文档；
* 提高表示质量：对于较长的文档，嵌入或其他表示的质量可能会下降，因为它们试图捕获太多信息。拆分可以使每个部分的表示更加集中和准确；
* 提高检索精度：在信息检索系统中，拆分可以提高搜索结果的粒度，使得查询与相关文档部分更精确地匹配；
* 优化计算资源：处理较小的文本块可以更节省内存，并允许更好地并行化处理任务。

可以在这个网页玩一玩 [ChunkViz](https://chunkviz.up.railway.app/)，一种可视化 splitter 的效果图，可以看到我们调整块大小，从而获得了不同的内容。

In [18]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI

# 我们仍旧使用 GPT-4o，这是一个支持多模态的 LLM
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.0,  # 让我们的 LLM 最准确
    max_tokens=2048,
    timeout=None,
    max_retries=2,
)

LangChain 给我们提供了很多种方式进行文件的读取 [Document loaders](https://python.langchain.com/docs/how_to/#document-loaders)，包含着 PDF、web pages、CSV、JSON 等等。同时 LangChain 也提供了部分平台的数据接口，可以参考[官方文档](https://python.langchain.com/docs/integrations/document_loaders/)。

In [56]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", chunk_size=100, chunk_overlap=0
)

# 读取一整个
docs = PyPDFLoader("./documents/ECE208_homework_tutorial.pdf").load()
texts = text_splitter.split_documents(
    docs
)

for index, text in enumerate(texts):
    print(f"===================Chunk {index}====================")
    print(text.page_content) # 可以看到已经按照字符数进行了拆分

===================Chunk 0====================
ECE 208 Homework tutorial
WI-22
This is a short tutorial on how to use GitHub Classroom. It will cover some usage of GitHub
commands, but we recommend you to refer to other materials for learning how to better use GitHub.
1 Prepare
Login to the your GitHub account.
2 Access the homework
• Go to the canvas Assignments page and click on the link in the homework.
• Accept the assignment on the pop-up page.
• You are ready to go! Refresh the page, and your assignment repository should have been created
(this might take several minutes).
• Click on the link on the GitHub Classroom page, and you will see the homework repository. It
should be in this format:
3 Work on the homework
• Clone the homework repository:
Open terminal and type:git clone $github repo
For example,
git clone
Replace $id with the homework id (e.g. hw1) and$github user name with your GitHub user-
name.
• Feel free to use any Python IDE you are familiar or comfortable with.
4 

我们尝试一下另外一种方法，使用 `RecursiveCharacterTextSplitter` 这一个方法更加智能。我们可以利用这种固有结构来指导我们的拆分策略，创建保持自然语言流的拆分，在拆分中保持语义连贯性，并适应不同级别的文本粒度。

在使用 `CharacterTextSplitter(separator="\n", chunk_size=100)` 的时候，它会按换行 `\n` 强制切割，然后截取固定长度的 chunk，不管语义是否完整。

而使用 `RecursiveCharacterTextSplitter(chunk_size=100)` 的时候，它会保持较大的单元（例如段落）的完整性。如果一个单元超出了块大小，它就会移动到下一个级别（例如，句子）。如果有必要，这个过程会持续到单词级别。

官方文档请查阅：[CharacterTextSplitter](https://python.langchain.com/api_reference/text_splitters/character/langchain_text_splitters.character.CharacterTextSplitter.html)、[RecursiveCharacterTextSplitter](https://python.langchain.com/api_reference/text_splitters/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)

In [55]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4",
    chunk_size=100,
    chunk_overlap=0,
)

doc = PyPDFLoader("./documents/ECE208_homework_tutorial.pdf").load()
texts = text_splitter.split_text(
    doc[0].page_content
)

for index, text in enumerate(texts[0:]):
    print(f"===================Chunk {index}====================")
    print(text) # 可以看到已经按照字符数进行了拆分

===================Chunk 0====================
ECE 208 Homework tutorial
WI-22
This is a short tutorial on how to use GitHub Classroom. It will cover some usage of GitHub
commands, but we recommend you to refer to other materials for learning how to better use GitHub.
1 Prepare
Login to the your GitHub account.
2 Access the homework
• Go to the canvas Assignments page and click on the link in the homework.
• Accept the assignment on the pop-up page.
===================Chunk 1====================
• You are ready to go! Refresh the page, and your assignment repository should have been created
(this might take several minutes).
• Click on the link on the GitHub Classroom page, and you will see the homework repository. It
should be in this format:
3 Work on the homework
• Clone the homework repository:
Open terminal and type:git clone $github repo
For example,
git clone
===================Chunk 2====================
Replace $id with the homework id (e.g. hw1) and$github user name with your

在上方的代码我们看不出什么明显的区别。我们使用中文来看一看：

In [83]:
from langchain_core.documents import Document

text = """
### 朴素贝叶斯的要求是什么？
贝叶斯定理
特征条件独立假设
### 4. 朴素贝叶斯的优缺点？
优点： 对小规模数据表现很好，适合多分类任务，适合增量式训练。
缺点：对输入数据的表达形式很敏感（离散、连续，值极大极小之类的）。
### 5. 朴素贝叶斯与LR区别？
朴素贝叶斯是生成模型，根据已有样本进行贝叶斯估计学习出先验概率P(Y)和条件概率P(X|Y)，进而求出联合分布概率P(XY)，最后利用贝叶斯定理求解P(Y|X)，而LR是判别模型，根据极大化对数似然函数直接求出条件概率P(Y|X)
朴素贝叶斯是基于很强的**条件独立假设**（在已知分类Y的条件下，各个特征变量取值是相互独立的），而LR则对此没有要求
朴素贝叶斯适用于数据集少的情景，而LR适用于大规模数据集。
"""
doc = Document(page_content=text)

In [84]:
# 使用 CharacterTextSplitter（按换行切）
char_splitter = CharacterTextSplitter(separator="\n", chunk_size=50, chunk_overlap=0)
char_chunks = char_splitter.split_documents([doc])

# 使用 RecursiveCharacterTextSplitter（自动层级分割）
recursive_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=0)
recursive_chunks = recursive_splitter.split_documents([doc])

for index, text in enumerate(char_chunks):
    print(f"===================Char Splitter Chunk {index}====================")
    print(text) # 可以看到已经按照字符数进行了拆分

print("\n+++++++++++++++++++++++++++++++++++++++++++++++++++++\n")

for index, text in enumerate(recursive_chunks):
    print(f"===================Recursive Splitter Chunk {index}====================")
    print(text) # 可以看到已经按照字符数进行了拆分

Created a chunk of size 115, which is longer than the specified 50
Created a chunk of size 59, which is longer than the specified 50


===================Char Splitter Chunk 0====================
page_content='### 朴素贝叶斯的要求是什么？
贝叶斯定理
特征条件独立假设
### 4. 朴素贝叶斯的优缺点？'
===================Char Splitter Chunk 1====================
page_content='优点： 对小规模数据表现很好，适合多分类任务，适合增量式训练。'
===================Char Splitter Chunk 2====================
page_content='缺点：对输入数据的表达形式很敏感（离散、连续，值极大极小之类的）。'
===================Char Splitter Chunk 3====================
page_content='### 5. 朴素贝叶斯与LR区别？'
===================Char Splitter Chunk 4====================
page_content='朴素贝叶斯是生成模型，根据已有样本进行贝叶斯估计学习出先验概率P(Y)和条件概率P(X|Y)，进而求出联合分布概率P(XY)，最后利用贝叶斯定理求解P(Y|X)，而LR是判别模型，根据极大化对数似然函数直接求出条件概率P(Y|X)'
===================Char Splitter Chunk 5====================
page_content='朴素贝叶斯是基于很强的**条件独立假设**（在已知分类Y的条件下，各个特征变量取值是相互独立的），而LR则对此没有要求'
===================Char Splitter Chunk 6====================
page_content='朴素贝叶斯适用于数据集少的情景，而LR适用于大规模数据集。'

+++++++++++++++++++++++++++++++++++++++++++++++++++++

===================Recursive Splitter Chunk 0====================
page_

 实际上，`RecursiveCharacterTextSplitter` 是专门用来把 “大块文本智能切成小块” 的工具，它会尽量保留语义结构，在必要时再递归地切分。

可以看到在 Char Splitter Chunk 4 中一段话有非常长的内容，因为优先级的问题，如果没有出现 `\n`就不会切分，但是这样会让不同的文本长度跨越过大。但是在 Recursive Splitter Chunk 9 中就不会，根据提示 “Created a chunk of size 115, which is longer than the specified 50Created a chunk of size 115, which is longer than the specified 50”，实际上工具自动把我们的长度进行了递归的切分。

在 LangChain 中还有别的的文本切分工具，比如 [Markdown](https://python.langchain.com/docs/how_to/markdown_header_metadata_splitter/)、[JSON](https://python.langchain.com/docs/how_to/recursive_json_splitter/)、[代码](https://python.langchain.com/docs/how_to/code_splitter/)、[HTML](https://python.langchain.com/docs/how_to/split_html/)，好在官方已经提供了对于这些文本的拆分器，具体使用方法其实是与 text splitter 相一致的。除此之外，还有一个更加美好的方法，是基于语义分割，参考 Greg Kamradt 的[文档](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb)。